In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [41]:
#hyperparameters
batch_size = 8 # How many sentences are grouped together during training
block_size = 8 # Sentence Length with which one does training. Maximum Context length == Block Size
max_iters = 3000
eval_interval = 300
learning_rate = 1e-4
with open('books.txt', 'r', encoding= 'UTF -8') as f:
    text = f.read()
print("input read")


## Vocabulary in our context is the individual characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)

input read
156


In [42]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join(itos[l] for l in i)

In [43]:
df = pd.read_csv('dataframe_testing.csv')
df = df.drop(columns=['Link'])

In [56]:



def prepare_data(df):
    train_data = []

    for _, row in df.iterrows():
        encoder_input = f"{row['Title']} | {row['Author']} | {row['Bookshelf']}"
        decoder_input = row['Text']  # Take a chunk of the text

        enc_tokens = encode(encoder_input)
        dec_tokens = encode(decoder_input)

        train_data.append((torch.tensor(enc_tokens, dtype=torch.long), torch.tensor(dec_tokens, dtype=torch.long)))

    return train_data

tokenized_data = prepare_data(df)
data_len = len(tokenized_data)
print(len(tokenized_data))

n = int(0.8*data_len)
train_data = tokenized_data[:n]
val_data = tokenized_data[n:]

def get_batch_encoder_decoder(split):
    if split == 'train':
        data = train_data
    else:
        data = val_data
    data_idx = torch.randint(len(data), (1,))
    print(data_idx)
    encoder_input = data[data_idx.item()][0]
    ix = torch.randint(len(encoder_input)-block_size, (batch_size,))
    x = torch.stack([encoder_input[i:i+block_size] for i in ix])
    decoder_input = data[data_idx.item()][1]
    ix = torch.randint(len(decoder_input)-block_size, (batch_size,))
    y = torch.stack([decoder_input[i:i+block_size] for i in ix])
    return x, y

xb, yb = get_batch_encoder_decoder('val')
xb, yb


10
8 2
tensor([1])


(tensor([[88,  1, 91,  1, 32, 77, 72, 76],
         [81, 77, 64, 67, 64, 88,  1, 91],
         [91,  1, 54, 72, 75, 75, 72, 64],
         [77, 64, 67, 64, 88,  1, 91,  1],
         [68, 81, 72, 66, 64, 77,  1, 33],
         [82, 78, 77,  1, 91,  1, 54, 72],
         [72, 64, 76,  1, 51, 15,  1, 39],
         [ 1, 33, 72, 82, 78, 77,  1, 91]]),
 tensor([[78,  1, 72, 83,  1, 69, 78, 81],
         [81, 68, 66, 68, 72, 85, 68,  1],
         [84, 81, 68,  1, 72, 82,  1, 83],
         [68,  1, 83, 78,  1, 83, 71, 68],
         [77, 67,  1,  3, 71, 72, 67, 68],
         [68, 81, 88,  1, 78, 75, 67,  1],
         [64, 13,  1, 65, 88,  1, 32, 81],
         [72, 77, 70,  1, 68, 64, 82, 83]]))